# Assignment 4
## Group Members:
### Nils Dunlop, e-mail: gusdunlni@student.gu.se
### Francisco Alejandro Erazo Piza, e-mail: guserafr@student.gu.se
### Chukwudumebi Ubogu, e-mail: gusuboch@student.gu.se
***

### Task 1: A small linear regression example in PyTorch
***

In [69]:
import pandas as pd
import numpy as np
import torch
from torch import optim

data = pd.read_csv('a4_synthetic.csv')

X = data.drop(columns='y').to_numpy()
Y = data.y.to_numpy()

(X, Y)

(array([[ 1.76405235,  0.40015721],
        [ 0.97873798,  2.2408932 ],
        [ 1.86755799, -0.97727788],
        [ 0.95008842, -0.15135721],
        [-0.10321885,  0.4105985 ],
        [ 0.14404357,  1.45427351],
        [ 0.76103773,  0.12167502],
        [ 0.44386323,  0.33367433],
        [ 1.49407907, -0.20515826],
        [ 0.3130677 , -0.85409574],
        [-2.55298982,  0.6536186 ],
        [ 0.8644362 , -0.74216502],
        [ 2.26975462, -1.45436567],
        [ 0.04575852, -0.18718385],
        [ 1.53277921,  1.46935877],
        [ 0.15494743,  0.37816252],
        [-0.88778575, -1.98079647],
        [-0.34791215,  0.15634897],
        [ 1.23029068,  1.20237985],
        [-0.38732682, -0.30230275],
        [-1.04855297, -1.42001794],
        [-1.70627019,  1.9507754 ],
        [-0.50965218, -0.4380743 ],
        [-1.25279536,  0.77749036],
        [-1.61389785, -0.21274028],
        [-0.89546656,  0.3869025 ],
        [-0.51080514, -1.18063218],
        [-0.02818223,  0.428

In [70]:
np.random.seed(1)
torch.manual_seed(1)

w_init = np.random.normal(size=(2, 1))
b_init = np.random.normal(size=(1, 1))

# Declare the parameter tensors
w = torch.tensor(w_init, dtype=torch.float32, requires_grad=True)
b = torch.tensor(b_init, dtype=torch.float32, requires_grad=True)

eta = 1e-2
opt = optim.SGD([w, b], lr=eta)

for i in range(10):

    sum_err = 0

    for row in range(X.shape[0]):
        x = torch.tensor(X[row, :], dtype=torch.float32).view(1, -1)
        y = torch.tensor(Y[row], dtype=torch.float32).view(1, -1)

        # Forward pass.
        y_pred = x.mm(w) + b
        err = (y_pred - y).pow(2).sum()

        # Backward and update.
        opt.zero_grad()
        err.backward()
        opt.step()

        # For statistics.
        sum_err += err.item()

    mse = sum_err / X.shape[0]
    print(f'Epoch {i+1}: MSE =', mse)

Epoch 1: MSE = 0.7999662647869263
Epoch 2: MSE = 0.017392394159767264
Epoch 3: MSE = 0.009377418162580966
Epoch 4: MSE = 0.009355327616258364
Epoch 5: MSE = 0.009365440349979508
Epoch 6: MSE = 0.009366988411857164
Epoch 7: MSE = 0.009367207068114567
Epoch 8: MSE = 0.009367238481529512
Epoch 9: MSE = 0.009367244712136654
Epoch 10: MSE = 0.009367244620257224


### Task 2: Implementing tensor arithmetics for forward computations
***

In [71]:
class Tensor:

    # Constructor. Just store the input values.
    def __init__(self, data, requires_grad=False, grad_fn=None):
        self.data = data
        self.shape = data.shape
        self.grad_fn = grad_fn
        self.requires_grad = requires_grad
        self.grad = None

    # So that we can print the object or show it in a notebook cell.
    def __repr__(self):
        dstr = repr(self.data)
        if self.requires_grad:
            gstr = ', requires_grad=True'
        elif self.grad_fn is not None:
            gstr = f', grad_fn={self.grad_fn}'
        else:
            gstr = ''
        return f'Tensor({dstr}{gstr})'

    # Extract one numerical value from this tensor.
    def item(self):
        return self.data.item()

    # Operator +
    def __add__(self, right):
        # Call the helper function defined below.
        return addition(self, right)

    # Operator -
    def __sub__(self, right):
        return subtraction(self, right)

    # Operator @
    def __matmul__(self, right):
        return matmul(self, right)
    
    # Operator **
    def __pow__(self, right):
        # NOTE! We are assuming that right is an integer here, not a Tensor!
        if not isinstance(right, int):
            raise Exception('only integers allowed')
        if right < 2:
            raise Exception('power must be >= 2')
        return power(self, right)

    def backward(self, grad_output=None):
        if self.requires_grad:
            # Initialize grad_output if this is the end tensor
            if grad_output is None:
                grad_output = np.ones_like(self.data)

            # Accumulate gradients
            if self.grad is None:
                self.grad = grad_output
            else:
                self.grad += grad_output

            # Propagate gradients if this tensor is a result of an operation
            if self.grad_fn is not None:
                adjusted_grad_output = [self.grad_fn.compute_grad(grad_output, input_tensor) for input_tensor in self.grad_fn.inputs]

                for input_tensor, grad in zip(self.grad_fn.inputs, adjusted_grad_output):
                    input_tensor.backward(grad)

In [72]:
# Create the tensor object
def tensor(data, requires_grad=False):
    return Tensor(data, requires_grad)

# Update the helper functions to implement the various arithmetic operations.
def addition(left, right):
    new_data = left.data + right.data
    return Tensor(new_data, requires_grad=True, grad_fn=AdditionNode(left, right))

def subtraction(left, right):
    new_data = left.data - right.data
    return Tensor(new_data, requires_grad=True, grad_fn=SubtractionNode(left, right))

def matmul(left, right):
    new_data = left.data.dot(right.data)
    return Tensor(new_data, requires_grad=True, grad_fn=MatMulNode(left, right))

def power(left, exponent):
    new_data = left.data ** exponent
    return Tensor(new_data, requires_grad=True, grad_fn=PowerNode(left, exponent))

In [73]:
# Two tensors holding row vectors.
x1 = tensor(np.array([[2.0, 3.0]]))
x2 = tensor(np.array([[1.0, 4.0]]))
# A tensors holding a column vector.
w = tensor(np.array([[-1.0], [1.2]]))

# Test the arithmetic operations.
test_plus = x1 + x2
test_minus = x1 - x2
test_power = x2 ** 2
test_matmul = x1 @ w

print(f'Test of addition: {x1.data} + {x2.data} = {test_plus.data}')
print(f'Test of subtraction: {x1.data} - {x2.data} = {test_minus.data}')
print(f'Test of power: {x2.data} ** 2 = {test_power.data}')
print(f'Test of matrix multiplication: {x1.data} @ {w.data} = {test_matmul.data}')

# Check that the results are as expected. Will crash if there is a miscalculation.
assert(np.allclose(test_plus.data, np.array([[3.0, 7.0]])))
assert(np.allclose(test_minus.data, np.array([[1.0, -1.0]])))
assert(np.allclose(test_power.data, np.array([[1.0, 16.0]])))
assert(np.allclose(test_matmul.data, np.array([[1.6]])))

Test of addition: [[2. 3.]] + [[1. 4.]] = [[3. 7.]]
Test of subtraction: [[2. 3.]] - [[1. 4.]] = [[ 1. -1.]]
Test of power: [[1. 4.]] ** 2 = [[ 1. 16.]]
Test of matrix multiplication: [[2. 3.]] @ [[-1. ]
 [ 1.2]] = [[1.6]]


### Task 3: Building the computational graph
***

In [74]:
class Node:
    def __init__(self, *inputs):
        self.inputs = inputs

    def backward(self, grad_output):
        raise NotImplementedError('Unimplemented')

    def __repr__(self):
        return f"{self.__class__.__name__}(id={id(self)})"

class AdditionNode(Node):
    def __init__(self, left, right):
        super().__init__(left, right)
        self.left = left
        self.right = right

    def compute_grad(self, grad_output, input_tensor):
        # Gradient does not change for addition
        return grad_output

    def backward(self, grad_output):
        adjusted_grad_output_left = self.compute_grad(grad_output, self.inputs[0])
        adjusted_grad_output_right = self.compute_grad(grad_output, self.inputs[1])

        if self.inputs[0].requires_grad:
            self.inputs[0].backward(adjusted_grad_output_left)
        if self.inputs[1].requires_grad:
            self.inputs[1].backward(adjusted_grad_output_right)
        

class SubtractionNode(Node):
    def __init__(self, left, right):
        super().__init__(left, right)
        self.left = left
        self.right = right

    def compute_grad(self, grad_output, input_tensor):
        if input_tensor == self.inputs[1]:
            return -grad_output
        return grad_output

    def backward(self, grad_output):
        self.inputs[0].backward(self.compute_grad(grad_output, self.inputs[0]))
        self.inputs[1].backward(self.compute_grad(grad_output, self.inputs[1]))
        
class MatMulNode(Node):
    def __init__(self, left, right):
        super().__init__(left, right)
        self.left = left
        self.right = right


    def compute_grad(self, grad_output, input_tensor):
        if input_tensor == self.inputs[0]:
            return grad_output.dot(self.inputs[1].data.T)
        else:
            return self.inputs[0].data.T.dot(grad_output)

    def backward(self, grad_output):
        self.inputs[0].backward(self.compute_grad(grad_output, self.inputs[0]))
        self.inputs[1].backward(self.compute_grad(grad_output, self.inputs[1]))
    
class PowerNode(Node):
    def __init__(self, base, exponent):
        super().__init__(base)
        self.base = base
        self.exponent = exponent

    def compute_grad(self, grad_output, input_tensor):
        return grad_output * self.exponent * (input_tensor.data ** (self.exponent - 1))

    def backward(self, grad_output):
        adjusted_grad_output = self.compute_grad(grad_output, self.inputs[0])
        self.inputs[0].backward(adjusted_grad_output)

In [75]:
x = tensor(np.array([[2.0, 3.0]]))
w1 = tensor(np.array([[1.0, 4.0]]), requires_grad=True)
w2 = tensor(np.array([[3.0, -1.0]]), requires_grad=True)

test_graph = x + w1 + w2

print('Computational graph top node after x + w1 + w2:', test_graph.grad_fn)

assert(isinstance(test_graph.grad_fn, AdditionNode))
assert(test_graph.grad_fn.right is w2)
assert(test_graph.grad_fn.left.grad_fn.left is x)
assert(test_graph.grad_fn.left.grad_fn.right is w1)

Computational graph top node after x + w1 + w2: AdditionNode(id=2457500736992)


### Task 4: Implementing the backward computations
***

In [76]:
x = tensor(np.array([[2.0, 3.0]]))
w = tensor(np.array([[-1.0], [1.2]]), requires_grad=True)
y = tensor(np.array([[0.2]]))

# We could as well write simply loss = (x @ w - y)**2
# We break it down into steps here if you need to debug.

model_out = x @ w
diff = model_out - y
loss = diff ** 2

loss.backward()

print('Gradient of loss w.r.t. w =\n', w.grad)

assert(np.allclose(w.grad, np.array([[5.6], [8.4]])))
assert(x.grad is None)
assert(y.grad is None)

Gradient of loss w.r.t. w =
 [[5.6]
 [8.4]]


### Task 5: Optimizers to update the model parameters
***

In [77]:
class SGDOptimizer:
    def __init__(self, parameters, lr=0.01):
        self.parameters = parameters
        self.lr = lr

    def step(self):
        # Update each parameter
        for param in self.parameters:
            if param.grad is not None:
                param.data -= self.lr * param.grad

    def zero_grad(self):
        # Reset gradients for each parameter
        for param in self.parameters:
            if param.grad is not None:
                param.grad = np.zeros_like(param.grad)

### Questions and answers

- Q1: Do we have to implement gradient accumulation?
- Q2: Am I allowed to implement <some function>?
- Q3: In Task 5, what do you mean by "identical"? 